<a href="https://colab.research.google.com/github/vijayc9/Human-Activity-Recognition/blob/main/LSTM_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Applying LSTM Models on Raw Data**

In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2


In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def new_confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

    
   # result = confusion_matrix(Y_true, Y_pred)

    #plt.figure(figsize=(10, 8))
   # sns.heatmap(result, 
    #            xticklabels= list(ACTIVITIES.values()), 
     #           yticklabels=list(ACTIVITIES.values()), 
      #          annot=True, fmt="d");
   # plt.title("Confusion matrix")
   # plt.ylabel('True label')
   # plt.xlabel('Predicted label')
    plt.show()  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<html><h1><p style="color:red">Loading Data / Making Data </p></h1></html>

In [ ]:
# Data directory
DATADIR = '/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset'#'UCI_HAR_Dataset'


# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [ ]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [ ]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [ ]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


**1. Defining the Architecture of 1-Layer of LSTM**




In [ ]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 24s 47ms/step - loss: 1.2052 - accuracy: 0.4944 - val_loss: 0.9226 - val_accuracy: 0.5945
Epoch 2/30
460/460 [==============================] - 23s 49ms/step - loss: 0.8041 - accuracy: 0.6500 - val_loss: 0.7612 - val_accuracy: 0.6841
Epoch 3/30
460/460 [==============================] - 21s 45ms/step - loss: 0.6805 - accuracy: 0.7203 - val_loss: 0.6757 - val_accuracy: 0.7418
Epoch 4/30
460/460 [==============================] - 21s 46ms/step - loss: 0.5312 - accuracy: 0.8111 - val_loss: 0.7307 - val_accuracy: 0.7608
Epoch 5/30
460/460 [==============================] - 21s 45ms/step - loss: 0.3963 - accuracy: 0.8723 - val_loss: 0.4535 - val_accuracy: 0.8585
Epoch 6/30
460/460 [==============================] - 22s 48ms/step - loss: 0.3088 - accuracy: 0.9101 - val_loss: 0.4747 - val_accuracy: 0.8504
Epoch 7/30
460/460 [==============================] - 26s 57ms/step - loss: 0.2765 - accuracy: 0.9189 - val_loss: 0.4399 - val_accuracy:

In [ ]:
# Confusion Matrix
new_confusion_matrix(Y_test, model.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,27,0,0,0
SITTING,3,408,77,0,1,2
STANDING,0,83,447,1,0,1
WALKING,0,0,1,446,24,25
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,0,0,1,33,437


In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 14ms/step - loss: 0.6416 - accuracy: 0.9053

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.6415585875511169, 0.9053274393081665]


<html><h3><p style="color:#008080">With a simple 1 layer architecture we got 90.09% accuracy and a loss of 0.47</p></h4></html>

<html><h1><p style="color:red">2. Defining the Architecture of 2-Layer of LSTM with more hyperparameter tunning </p></h1></html>

### 2.1 First Model for 2-Layer of LSTM with more hyperparameter tunning

In [ ]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 48)           11136     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 48)           192       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 48)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                10368     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 21,894
Trainable params: 21,798
Non-trainable params: 96
_________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 84s 11ms/step - loss: 1.7780 - accuracy: 0.6234 - val_loss: 1.4911 - val_accuracy: 0.6071
Epoch 2/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.9271 - accuracy: 0.7723 - val_loss: 0.6301 - val_accuracy: 0.8151
Epoch 3/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.6144 - accuracy: 0.8328 - val_loss: 0.5123 - val_accuracy: 0.8185
Epoch 4/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.3950 - accuracy: 0.8885 - val_loss: 0.3122 - val_accuracy: 0.8806
Epoch 5/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.3201 - accuracy: 0.9059 - val_loss: 0.2809 - val_accuracy: 0.9009
Epoch 6/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.2534 - accuracy: 0.9246 - val_loss: 0.2453 - val_accuracy: 0.9230
Epoch 7/30
7352/7352 [==============================] - 85s 12ms/step - loss

In [ ]:
# Confusion Matrix
new_confusion_matrix(Y_test, model.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,374,113,0,0,4
STANDING,0,65,467,0,0,0
WALKING,0,0,0,445,50,1
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,0,0,2,49,420


In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

2947/2947 [==============================] - 3s 1ms/step

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.3922457374024961, 0.903630793094635]


### 2.2 Second Model for 2-Layer of LSTM with more hyperparameter tunning

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 64)           256       
_________________________________________________________________
dropout_4 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 48)                21696     
_________________________________________________________________
dropout_5 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 294       
Total params: 41,190
Trainable params: 41,062
Non-trainable params: 128
________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 85s 11ms/step - loss: 1.8590 - accuracy: 0.6396 - val_loss: 1.3335 - val_accuracy: 0.6362
Epoch 2/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.9453 - accuracy: 0.8044 - val_loss: 0.7366 - val_accuracy: 0.8378
Epoch 3/30
7352/7352 [==============================] - 85s 11ms/step - loss: 0.4631 - accuracy: 0.8964 - val_loss: 0.3560 - val_accuracy: 0.8890
Epoch 4/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.2618 - accuracy: 0.9229 - val_loss: 0.2307 - val_accuracy: 0.9240
Epoch 5/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.2229 - accuracy: 0.9306 - val_loss: 0.2192 - val_accuracy: 0.9182
Epoch 6/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.2097 - accuracy: 0.9287 - val_loss: 0.2898 - val_accuracy: 0.9125
Epoch 7/30
7352/7352 [==============================] - 85s 12ms/step - loss

In [ ]:
# Confusion Matrix
new_confusion_matrix(Y_test, model.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,519,0,18,0,0,0
SITTING,7,388,96,0,0,0
STANDING,0,88,444,0,0,0
WALKING,0,0,0,477,16,3
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,0,0,6,10,455


In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

2947/2947 [==============================] - 3s 1ms/step

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.279539362433966, 0.917203962802887]


**Further extention**
- 1-Layer LSTM Layer with hidden layer = 128 with dropout = 0.5
- 1-Layer LSTM Layer with hidden layer = 324 with slight change in dropout = 0.6
- 2-Layer LSTM Layer with hidden layer, h1 = 256 & h2 = 128 with dropout 0.7 & 0.6 respectively
- 2-Layer LSTM Layer with hidden layer, h1 = 128 & h2 = 64 with dropout 0.2 & 0.5 respectively.

###############

for last model: archi...

epochs1 = 30
batch_size1= 32
n_hidden1 = 128
n_hidden2 =64

######################################

# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden1, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation

model.add(LSTM(n_hidden2, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='sigmoid'))


# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

##############################

# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

score = model.evaluate(X_test, Y_test)
score

############################
import matplotlib.pyplot as plt
%matplotlib inline
print('Test score:', scores[0]) 
print('Test accuracy:', scores[1])
 
fig,ax = plt.subplots(1,1) 
ax.set_xlabel('epoch') 
ax.set_ylabel('Categorical Crossentropy Loss')
 
# list of epoch numbers 
x = list(range(1,epochs+1))

vy = history.history['val_loss'] 
ty = history.history['loss'] 
plt_dynamic(x, vy, ty, ax)